In [1]:
# Forget why I need this. Folder creation?


import os 

 # Used to parse webpages
from requests import Response
from bs4 import BeautifulSoup as bs4
from bs4 import element       as bel
import re

# Used for asynchronous webpage fetching
import concurrent.futures as cof 
from requests_futures.sessions import FuturesSession

# Used for a good-looking display/diagnostic system 
from IPython import display as dis
import ipywidgets as ipw

# Used for delays and tracking execution time
import time


In [2]:
import sys

In [3]:
sys.version

'3.7.1 (default, Oct 22 2018, 11:21:55) \n[GCC 8.2.0]'

### Automatic parsing for webpage
Fetches the apropriate table, and parses it for GUID/material name pairs for later use. 
Set up as two functions, that run as a post-processing step for the 

In [72]:
MatDict = dict()
class regexes:
    def __init__(self):
        self.tagloc=re.compile(r'.*Replacing.*')
regexes = regexes()

pages_scanned_kludge = list()
output_dict_kludge = dict()

def MatWebTableFetch (thispage,**kwargs):
    pagenum = thispage.url.split("=")[1]
    if (thispage.status_code != 200):
        pages_scanned_kludge[int(pagenum)-1].button_style='warning'
        print(f"Warning! URL {thispage.url} returned code {thispage.status_code} ")
    else:
        pages_scanned_kludge[int(pagenum)-1].button_style='success'
        
    PageHTML = bs4(thispage.text,'html.parser')
    Tagloc = PageHTML.find_all(text=r.tagloc)
    if len(Tagloc) != 1:
        raise ValueError("Did not find single tag locator. Found {0} of them".format(len(Tagloc)))
    Tagloc = Tagloc[0]
    FetchedTable = Tagloc.find_all_next('ul')
    if len(FetchedTable) != 1:
        raise ValueError("Did not find a single table. Found {0} of them".format(len(FetchedTable)))
    FetchedTable = FetchedTable[0]
    return (FetchedTable.find_all("li"),pagenum)


def GetMatNameGUID (kludge_tuple,**kwargs):
    listoflitags = kludge_tuple[0]
    pagenum = kludge_tuple[1]
    outputdict= output_dict_kludge
    if len(listoflitags) == 0:
        pages_scanned_kludge
        
    for item in listoflitags:
        # The URL contains a matguid value. This must be extracted from the rest
        extractedURL  = str(item.a["href"])
        GUIDstring = extractedURL.split("matguid=", 1)[1]
        extractedGUID = int(GUIDstring,16)
        # The material name is formatted using <sub> tags. These must be removed.
        # I replace them with LaTeX syntax.
        matname = str()
        for child in item.a.children:
            if type(child) is bel.NavigableString :
                
                matname+=str(child)
            if type(child) is bel.Tag:
                if child.name == 'sub':
                    if child.string is None:
                        raise ValueError("Unable to parse "+str(child))
                    matname+=("_{"+child.string+"}")
        outputdict[extractedGUID] = matname
    
    return len(listoflitags)

def debugHookPassedData(hook,**kwargs):
    print(f"Argument: {hook} \n kwargs:{kwargs}")



In [5]:
class displayobject:
    def setval(self,index,value):
        print(f"page {index} announces status {value}")


        
# Probably best to modify this at some point into a child of the requests.response
# class. A lot of work-around-variables are being added.
class scraper_return:
    __slots__ = ['pipedata','starttime','cputime','realtime', 'displayobject',
                    'fetchcode','pagenumber','len', 'attempts']
    def __init__(self):
        
        self.starttime = time.time()
        
        # Time is the results of timing tests run on the process
        # Here, I'm recording both CPU time and net time
        self.cputime = 0
        self.realtime = 0
        
        # This is the http return code value received when the
        # webpage was retreived.
        self.fetchcode = 0
        
        # This is the pagenumber that was requested. It can be
        # used to construct the URL
        self.pagenumber = -1
        
        # A object with a "setval" method taking two arguments:
        # A index, and a value to set it to. 
        self.displayobject = displayobject()
        
        # This is the length (number of entries)
        # discovered as valid on the page
        self.len = 0
        
        # this is the number of times the system tried
        # before getting results.
        self.attempts = 0
scraper_return()
        

In [6]:
class button_grid(displayobject):
    def __init__(self,iterator):
        self.buttonlist = dict()
        for i in iterator:
            newbut = ipw.Button(value=True,
                                description=str(i), 
                                layout=ipw.Layout(width='42px', height='25px'),
                                disabled=True)
            self.buttonlist[i]=newbut
        container=ipw.Box(tuple(self.buttonlist.values()))
        container.layout.flex_flow = "row wrap"
        display(container)
    def setval (self, index, value):
        states = {'running':'turquoise','bad_status':'orange',
                  'short_len':'maroon','zero_len':'red','done':'lightgreen'}
        self.buttonlist[index].style.button_color = states[value]
        


In [7]:
button_grid(range(1,10))

Box(children=(Button(description='1', disabled=True, layout=Layout(height='25px', width='42px'), style=ButtonS…

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_adjust_thread_count', '_broken', '_counter', '_initargs', '_initializer', '_initializer_failed', '_max_workers', '_shutdown', '_shutdown_lock', '_thread_name_prefix', '_threads', '_work_queue', 'map', 'shutdown', 'submit']
<class 'tuple'>


In [73]:

class GUID_scraper (FuturesSession):
    ########## STRUCTURE OF CLASS (TODO) ########
    ## Idealy, GUID_scraper is a pipeline.
    ## A mutable variable is generated by process_page, 
    ## and it is passed through the request function
    ## and every hook, with the required data added to
    ## it along the way. 
    ## Due to the way that hooks work in this, this means
    ## idealy, the hooks would all return void, making no
    ## changes to the container, just modifying the contents.
    
    ## It's closer than it was when it was purely functional,
    ## but still needs a lot of cleaning up for this.
    ## The class passed down the line would very likely 
    ## inherit from requests.Response, to assure compatibility
    ## with the requests hook structure it must pass through.
    response = scraper_return()
    

    
 
        
    
    #### Extracts the list of materials and ids from the webpage. 
    def MatWebTableFetch (self,thispage,**kwargs):
            pagenum = thispage.response.pagenumber
            if (thispage.status_code != 200):
                thispage.response.displayobject.setval(response.pagenumber,'bad_status')
            
            PageHTML = bs4(thispage.text,'html.parser')
            Tagloc = PageHTML.find_all(text=regexes.tagloc)
            if len(Tagloc) != 1:
                raise ValueError("Did not find single tag locator. Found {0} of them".format(len(Tagloc)))
            Tagloc = Tagloc[0]
            FetchedTable = Tagloc.find_all_next('ul')
            if len(FetchedTable) != 1:
                raise ValueError("Did not find a single table. Found {0} of them".format(len(FetchedTable)))
            FetchedTable = FetchedTable[0]
            thispage.response.pipedata = FetchedTable.find_all("li")
            return thispage
   
    #### Extracts the entries from the HTML formatted list
    def GetMatNameGUID (self,page,**kwargs):
            #print("GetMatNameGUID got",page)
            listoflitags = page.response.pipedata
            pagenum = page.response.pagenumber
            page.response.len = len(listoflitags)
            outputdict = dict()
            #print("detected",len(listoflitags))
            if page.response.len == 0:
                page.response.displayobject.setval(pagenum,'zero_len')
            elif page.response.len < 250:
                page.response.displayobject.setval(pagenum,'short_len')

            for item in listoflitags:
                # The URL contains a matguid value. This must be extracted from the rest
                extractedURL  = str(item.a["href"])
                GUIDstring = extractedURL.split("matguid=", 1)[1]
                extractedGUID = int(GUIDstring,16)
                # The material name is formatted using <sub> tags. These must be removed.
                # I replace them with LaTeX syntax.
                matname = str()
                for child in item.a.children:
                    if type(child) is bel.NavigableString :

                        matname+=str(child)
                    if type(child) is bel.Tag:
                        if child.name == 'sub':
                            if child.string is None:
                                raise ValueError("Unable to parse "+str(child))
                            matname+=("_{"+child.string+"}")
                outputdict[extractedGUID] = matname
            page.response.pipedata = outputdict
            return page
        
    #### Does the final tasks needed. 
    #### When I add auto-retry, this function will also be responsible for confirming
    #### that the page has been downloaded correctly, and scheduling a retry if it 
    #### has not. 
    def cleanup(self,page,**kwargs):
        #print(page.response.len, "<resp stat>", page.status_code)
        if page.response.len >= 250 and page.status_code == 200:
            page.response.displayobject.setval(page.response.pagenumber,'done')
            
    
        
        
        
    #### A way to request pages with just a page number and response statement.
    #### Essentialy just a wrapper for the request function that does the hard work.
    def process_page(self, pagenum, response = None, **kwargs):
        if response is None:
            response = scraper_return()
            response.pagenumber = pagenum

        return self.request("GET","http://www.matweb.com/search/GetAllMatls.aspx",
                     hooks=kwargs.get('hooks',dict()), 
                     params={"p":response.pagenumber}, 
                     response = response,
                     **kwargs)
    
    # TODO switch to real parameters here,
    # args, kwargs isn't helpful
    
    #### the request function is called to grab stuff from the web.
    #### by running my own code, before simpily calling the base-class's
    #### request function, I can start the timers that I need to make everything
    #### else work.
    def request(self,*args, **kwargs):
        hooks     = kwargs.get('hooks',dict())
        response  = kwargs.pop('response')
        #print(response.displayobject)
        response.starttime = time.time()
        response.cputime = time.thread_time_ns()
        response.realtime = time.perf_counter_ns()
        
        ######### HELPER FUNCTIONS ###########
        #### This is somewhat of a workaround. 
        #### the 'response', or 'scraper_return' concept is that from
        #### start to finish, you're passing the same sort of object.
        #### The pipeline speaks the same language, so you can swap out
        #### the steps freely. I started by trying to implement a lightweight
        #### container class (scraper_return), but I ran into problems with
        #### the clean-up code after the hooks ran realy, realy wanting a
        #### requests.Response object. So I just threw a response object in
        #### there with it. this can likely be neatened up further with a real
        #### response subclass with the features I wand, but it's alright for now.
        def hook_resp_merge(hookdata,**kwargs):
            response.cputime = time.thread_time_ns()-response.cputime
            response.realtime = time.perf_counter_ns()-response.realtime
            response.attempts += 1
            hookdata.response = response
            return hookdata
        ###### END OF HELPER FUNCTIONS #######
        

        # Helps work with the fact that hooks can already exist as a sequence of functions,
        # already exist as a single function, or not exist at all before the three hooks are
        # added to the end. this statement safely adds the hooks in the apropriate ordrer 
        # no matter what. 
        order_of_hooks = [hook_resp_merge,self.MatWebTableFetch,
                          self.GetMatNameGUID, self.cleanup]
        try:
            if isinstance(hooks['response'], (list, tuple)):
                # needs to be first so we don't time other hooks execution

                order_of_hooks.extend(hooks['response'])
                hooks['response'] = order_of_hooks
            else:
                order_of_hooks.append(hooks['response'])
                hooks['response'] = order_of_hooks
        except KeyError:
            hooks['response'] = order_of_hooks
        
        
        return super(GUID_scraper,self)\
               .request(*args,**kwargs)
        
         
        
            

In [77]:


responses = list()
allmatsURL = "http://www.matweb.com/search/GetAllMatls.aspx"
#pages_scanned_kludge = createnbuttons(522-1)
#print(Download.process_page(1).result().response.pipedata)

status = button_grid(range(1,522))
print(dir(status))

Download = GUID_scraper(max_workers=15)
#Download.hooks['response']=debugHookPassedData

futures = list()
for i in range(1,522):
    scraper = scraper_return()
    scraper.pagenumber = i
    scraper.displayobject = status
    futures.append(Download.process_page(i,scraper))

materials = dict()
results = list()

for completed in cof.as_completed(futures):
    if completed.exception():
        print(completed.result())
    response = completed.result().response
    materials.update(response.pipedata)
    response.pipedata = None
    results.append(response)
    
    


    

Box(children=(Button(description='1', disabled=True, layout=Layout(height='25px', width='42px'), style=ButtonS…

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'buttonlist', 'setval']


ValueError: Did not find single tag locator. Found 0 of them

In [78]:
print(len(materials))

4250


In [76]:
with open('materialdump.txt','w') as data:
    for key,entry in materials.items():
        data.write(f"{hex(key)} , {entry}")

2250

In [ ]:
"""
counter_value = 0
for key, value in output_dict_kludge.items():
    print(f"{value} is at location {key}")
    counter_value += 1
    if (counter_value > 100):
        break
"""

In [ ]:
materials = dict()
for response in responses:
    num = GetMatNameGUID(response.result(),materials)
    if num != 250:
        print(f"added {num} entries")
print(f"materials dict now has {len(materials)} entries")